In [ ]:
%run common_functions.ipynb
%run plotting_functions.ipynb

In [ ]:
import numpy as np
from scipy import stats 
from scipy.io import loadmat, savemat
import math
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from scipy.stats import zscore, pearsonr
import h5py
from random import sample

In [ ]:
data_path = '../data_hindbrain/'
with open(data_path + 'df_F6T07.pkl', 'rb') as pickle_in:
    df_F6T07 = pickle.load(pickle_in)  


pval = 0.01

n_lags = 3  # number of lags for GC
background = df_F6T07.background  # background for plotting

cell_centers = df_F6T07.cell_centers  # position of cells on the background
n_cells = len(cell_centers)  # number of cells
n_pairs = n_cells * (n_cells - 1)

swim_neurons = df_F6T07.swim_neurons  # index of motor-correlated neurons
n_cells_swim = len(swim_neurons)  # number of motor-correlated neurons
n_pairs_swim = n_cells_swim * (n_cells_swim - 1)

medial_neurons = df_F6T07.small_rect_neurons  # index of motor-correlated neurons in the medial region
n_cells_med = len(medial_neurons)  # number of motor-correlated neurons in the medial region
n_pairs_med = n_cells_med * (n_cells_med - 1)

fish = 6
trace = '07'
t_cycles = [round(5.81*(5+i*15)) for i in range(20)]  # start times of stimulus epochs

n_timesteps = t_cycles[-1] - t_cycles[0]  # 1656 timesteps corresponding to the 19 stimulus epochs
                                          # not 1744 that is the length of the whole recording
                                          # we keep 1656 for original GC to be coherent with shuffled GC

In [ ]:
# BVGC
dfn = n_lags
dfd_BV = n_timesteps - 3 * n_lags - 1  # 2 cells involved + 1

# For other GCs
# dfd_cBV = n_timesteps - 4 * n_lags - 1  # 2 cells involved + stimulus + 1
# dfd_MV = n_timesteps - (n_cells_med + 1) * n_lags - 1  # n_cells_med involved +1
# dfd_cMV = n_timesteps - (n_cells_med + 2) * n_lags - 1  # n_cells_med involved + stimulus + 1

threshold_F_ori = stats.f.ppf(1 - pval/n_pairs_med, dfn, dfd_BV) # Bonferroni corrected, common for all pairs

signals = df_F6T07.fzscored[t_cycles[0]:t_cycles[-1]]  # only use 1656 timesteps

nmc = 1000  # number of random shuffles - 100 should be enough

all_gcs_sh = np.zeros([n_cells_med, n_cells_med, nmc])  # GC matrices for each shuffle
all_fstats_sh = np.zeros([n_cells_med, n_cells_med, nmc])  # Fstats matrices for each shuffle
gc = np.zeros([n_cells_med, n_cells_med])  # original GC matrix
fstats = np.zeros([n_cells_med, n_cells_med])  # original Fstats matrix

for i, neuron1 in enumerate(medial_neurons):
    for j, neuron2 in enumerate(medial_neurons):
        if i != j: 
            # original GC analysis
            # /!\ NB: threshold_F is not Bonferroni corrected /!\ (but not used here)
            # GC_sig, GC, Fstat, threshold_F = bvgc(...)
            _, GC, Fstat, _ = bvgc(signal1, signal2, n_lags=n_lags, pval=0.01, tau=1, verbose=False)
            gc[i][j] = GC
            fstats[i][j] = Fstat
            
            # Shuffle nmc times            
            for n in range(nmc):
                # BVGC
                signal1 = shuffle_signal(signals[neuron1], t_cycles)  # shuffle driving signal
                signal2 = signals[neuron2][t_cycles[0]:t_cycles[-1]]
                _, GC, Fstat, _ = bvgc(signal1, signal2, n_lags=n_lags, pval=0.01, tau=1, verbose=False)

                all_gcs_sh[i][j][n] = GC
                all_fstats_sh[i][j][n] = Fstat


In [ ]:
# rescale
mean_F_sh = np.mean(all_fstats_sh, axis=2)

all_fstats_sh_rescaled = np.zeros([n_cells_med, n_cells_med, nmc])
all_fstats_sh_rescaled = all_fstats_sh / mean_F_sh  # follows F distribution



# customized threshold
threshold_F_new = mean_F_sh * threshold_F_ori
fstats_normalized = fstats / threshold_F_new

# normalized GC
GC_sig_new_thresh = np.zeros([n_cells_med, n_cells_med]) # normalized


# For hindbrain:
#     t_regr = n_timesteps - n_lags = 1653 because 1656 timesteps taken not 1744
#     BVGC
#         dof_full = 2 * n_lags = 6
#         dof_reduced = n_lags = 3
#     cBVGC
#         dof_full = 3 * n_lags = 9
#         dof_reduced = 2 * n_lags = 6
#     MVGC
#         dof_full = n_rois * n_lags = 60
#         dof_reduced = (n_rois - 1) * n_lags = 57
#     cMVGC
#         dof_full =  (n_rois + 1) * n_lags = 63
#         dof_reduced = n_rois * n_lags = 60
    
for i in range(n_cells_med):
    for j in range(n_cells_med):
        if fstats[i,j] > threshold_F_new[i,j]:
            normalized_GC_sig_new_thresh[i,j] = get_GC_from_Fstat(fstats_normalized[i,j], 2*n_lags, n_lags, t_regr)
        else:
            normalized_GC_sig_new_thresh[i,j] = np.nan
            